In [1]:
from PIL import Image
from torchvision import models
import pandas as pd
from torchvision import transforms as T 
import sys
import torch
import cv2 as cv
import numpy as np

/home/ashish/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
!pip3 install deep-sort-realtime

     |████████████████████████████████| 8.4 MB 708 kB/s eta 0:00:011     |██████████████████████████▏     | 6.9 MB 203 kB/s eta 0:00:08


In [4]:
from deep_sort_realtime.deepsort_tracker import DeepSort

In [5]:
data_path = './data/video/vid1.mp4'
out_path = './output/out.mp4'

In [6]:
faster_rcnn = models.detection.fasterrcnn_resnet50_fpn_v2(pretrained = True).to(device)

/home/ashish/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ashish/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_V2_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth" to /home/ashish/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth
100%|██████████| 167M/167M [02:27<00:00, 1.18MB/s] 


In [156]:
def bboxfrcnn(datapath, fasterrcnn_model, out_path, device):

    fasterrcnn_model.eval()
    vidc = cv.VideoCapture(datapath)

    fwidth = vidc.get(3)
    fheight = vidc.get(4)
    fwidth = int(fwidth)
    fheight = int(fheight)

    fps = vidc.get(cv.CAP_PROP_FPS)

    size = (fwidth, fheight)


    result = cv.VideoWriter(out_path, cv.VideoWriter_fourcc(*'MP4V'), fps, size)

    # call deepsort tracker.
    tracker = DeepSort(max_age = 30,n_init = 6,nms_max_overlap = 0,gating_only_position = True)


    track_bbs_ids = np.array([])
    f_no = 1
    
    cntr = 0
    cntr_lim = 1
    
    conf_thresh = 0.9
    
    while vidc.isOpened() and cntr < cntr_lim:
        ret, frame = vidc.read()
        if ret == True:
            transform = T.ToTensor()
            img = transform(frame).to(device)
            with torch.no_grad():
                pred = fasterrcnn_model([img])
                
    
            bounding_box,labels,confidence = pred[0]["boxes"].to('cpu'), pred[0]["labels"].to('cpu'), pred[0]["scores"].to('cpu')
            indices = torch.argwhere(confidence > conf_thresh).to('cpu').numpy()
            bounding_box, labels, confidence = bounding_box.numpy(), labels.numpy(), confidence.numpy()
            font = cv.FONT_HERSHEY_SIMPLEX
            
            
            arg_list = []
            for ind in indices:
                rl_ind = ind[0]
                if labels[rl_ind] == 3:
                    tup = (bounding_box[rl_ind],labels[rl_ind],confidence[rl_ind])
                    arg_list.append(tup)
            tracker.update_tracks(arg_list, frame=frame)
            
            
                
                
    
            if(len(indices)!=0):
                bbx_car = np.array([bounding_box[indices[i]] for i in range(len(indices)) if labels[indices[i]] == 3])
                bbx_car = bbx_car.reshape([bbx_car.shape[0], 4])
                for i in range(len(indices)):
                    if(labels[indices[i]] == 3):
                        x1,y1,x2,y2 = (bounding_box[indices[i]]).astype("int")[0]
                        class_name = "car"
                        cv.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),1)
                        cv.putText(frame,class_name,(x1-10,y1),font,1,(255,0,0), 3)
                
                result.write(frame)
        else:
            break
        f_no+=1

    return tracker.tracker._next_id - 1, track_bbs_ids


In [159]:
ids = bboxfrcnn(datapath=data_path, fasterrcnn_model=faster_rcnn, out_path=out_path, device=device)
ids[0]

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


8